## o-viton
## appearance generation
### train

#### 필요한 패키지 설치

In [1]:
import os
import sys

IS_COLAB = False

if 'COLAB_GPU' in os.environ:
    IS_COLAB=True
else:
    IS_COLAB=False

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install dominate
!pip install av
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 37.2MB 1.2MB/s 
     |████████████████████████████████| 276kB 12.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=a5fd9bb454035858f992dfd933bee767f5d945da163a72246cc69c1b60c9942d
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# console에서 다음 작업 진행할 것

!git clone https://github.com/wonyangcho/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data.git /content/o-viton 
# !cd o-viton
# !git checkout develop

Cloning into '/content/o-viton'...
remote: Enumerating objects: 644, done.
remote: Counting objects: 100% (644/644), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 1381 (delta 334), reused 423 (delta 141), pack-reused 737
Receiving objects: 100% (1381/1381), 10.56 MiB | 12.11 MiB/s, done.
Resolving deltas: 100% (715/715), done.


In [5]:
if IS_COLAB == True:
    from google.colab import drive

In [6]:
!nvidia-smi

Thu Feb  4 11:39:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from pathlib import Path

my_file = Path("/content/dataset")
if my_file.is_dir() == False:
  !wget -O o-viton-dataset.zip https://www.dropbox.com/s/b5ravd3x47sw5tx/o-viton-dataset_train.zip?dl=0
  !unzip -o -q o-viton-dataset.zip 
  !rm o-viton-dataset.zip
!ls ./dataset


--2021-02-04 11:39:31--  https://www.dropbox.com/s/b5ravd3x47sw5tx/o-viton-dataset_train.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b5ravd3x47sw5tx/o-viton-dataset_train.zip [following]
--2021-02-04 11:39:32--  https://www.dropbox.com/s/raw/b5ravd3x47sw5tx/o-viton-dataset_train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce8e4a5f90eb32463530dddfcb9.dl.dropboxusercontent.com/cd/0/inline/BIQwUqfMeTH_ez7-cyy7RKjJuMpA8Sg7R2fR0j7ZcyLWrhypaHWEEAARopkXxQFJmpzr4opZDsiAMnJ7HEteeE43nw-LTSscbRPx_8NaXT0Js1hvf7HRyE7Ov6lGjv1etSE/file# [following]
--2021-02-04 11:39:32--  https://uce8e4a5f90eb32463530dddfcb9.dl.dropboxusercontent.com/cd/0/inline/BIQwUqfMeTH_ez7-cyy7RKjJuMpA8Sg7R2fR0j7ZcyLWrhypaHWEEAARopkXxQFJ

#### google drive 연결

In [8]:
if IS_COLAB == True:
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/02_o-viton/"

Mounted at /content/drive
source


#### 관련 모듈 불러오기 

#### PATH

In [9]:
if IS_COLAB == True:
  sys.path.append("/content/drive/My Drive/02_o-viton/source/appearance_generation")
  sys.path.append("/content/o-viton/appearance_generation")


In [10]:
import torch
torch.__version__

'1.7.0+cu101'

In [11]:
from options.train_options import TrainOptions
from util.visualizer import Visualizer
import util.util as util
from models.models import create_model
from data.ov_train_dataset import RegularDataset
import time
import os
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
from collections import OrderedDict
from subprocess import call
import fractions
from torch.utils.tensorboard import SummaryWriter
from multiprocessing import Process
import matplotlib

%matplotlib inline

#### chekcpoint경로 확인

In [12]:
!ls /content/drive/MyDrive/02_o-viton/source/appearance_generation/checkpoints

vfr


#### main

In [13]:
%time
def lcm(a, b): return abs(a * b)/fractions.gcd(a, b) if a and b else 0

def main():
  opt = TrainOptions().parse(save=False)

  opt.print_freq = lcm(opt.print_freq, opt.batchSize)
  opt.checkpoints_dir = "/content/drive/My Drive/02_o-viton/source/appearance_generation/checkpoints"
  opt.dataroot = "/content/dataset"
  opt.continue_train = True #이전 checkpoint부터 학습을 하고 싶으면 True.
  opt.batchSize = 10
  opt.niter = 200
  opt.niter_decay = 0
 
  
  if opt.debug:
      opt.display_freq = 1
      opt.print_freq = 1
      opt.niter = 1
      opt.niter_decay = 0
      opt.max_dataset_size = 10

  

  iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')
  if opt.continue_train:
      try:
          start_epoch, epoch_iter = np.loadtxt(
              iter_path, delimiter=',', dtype=int)
      except:
          start_epoch, epoch_iter = 1, 0
      print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))
  else:
      start_epoch, epoch_iter = 1, 0



  # NEW DATALOADER
  augment = {}
  augment['1'] = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize((0.5, ), (0.5, ))])

  # augment['1'] = transforms.Compose(
  #     [
  #         transforms.ToTensor()])

  train_dataset = RegularDataset(opt, augment)

  train_dataloader = DataLoader(train_dataset,
                                batch_size=opt.batchSize,
                                shuffle=True,
                                num_workers=int(opt.nThreads),
                                pin_memory=True)

  # FOR DEBUGGING
  print(" #Checking  the dimension and type of data")
  for key in train_dataset[0].keys():
      try:
          x = train_dataset[0][key]
          print("name of the input and shape -- > ", key, x.shape)
          print("type,dtype,and min max -- >", type(x),
                x.dtype, torch.min(x), torch.max(x))
      except Exception as e:
          print("name of the input -- > ", key, train_dataset[0][key])
      print('----------------')

  dataset_size = len(train_dataset)
  print('#training images = %d' % dataset_size)

  # Initialize Networks
  model = create_model(opt)

  # Training Visualizer
  visualizer = Visualizer(opt)

  # Optimizers
  optimizer_G, optimizer_D = model.module.optimizer_G, model.module.optimizer_D

  # Train related additional details
  total_steps = (start_epoch-1) * dataset_size + epoch_iter
  display_delta = total_steps % opt.display_freq
  print_delta = total_steps % opt.print_freq
  save_delta = total_steps % opt.save_latest_freq

  for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
      epoch_start_time = time.time()
      if epoch != start_epoch:
          epoch_iter = epoch_iter % dataset_size
      for i, data in enumerate(train_dataloader, start=epoch_iter):
          if total_steps % opt.print_freq == print_delta:
              iter_start_time = time.time()
          total_steps += opt.batchSize
          epoch_iter += opt.batchSize

          # whether to collect output images
          save_fake = total_steps % opt.display_freq == display_delta

          ############## Forward Pass ######################
          # Reference Purpose
          # input_dict = {'seg_map': A_tensor, 'dense_map': dense_img, 'target': B_tensor, 'seg_map_path': A_path,
          # 'target_path': A_path, 'densepose_path': dense_path }
          # print( data['seg_mask'].shape)
          losses, generated = model(
              data['seg_map'], data['target'], data['seg_mask'], infer=save_fake)

          # sum per device losses
          losses = [torch.mean(x) if not isinstance(x, int)
                    else x for x in losses]
          loss_dict = dict(zip(model.module.loss_names, losses))

          # calculate final loss scalar
          loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
          loss_G = loss_dict['G_GAN'] + loss_dict.get('G_VGG', 0)

          ############### Backward Pass ####################
          # update generator weights
          optimizer_G.zero_grad()
          loss_G.backward()
          optimizer_G.step()

          # update discriminator weights
          optimizer_D.zero_grad()
          loss_D.backward()
          optimizer_D.step()

          ############## Display results and errors ##########
          # print out errors
          if total_steps % opt.print_freq == print_delta:
              errors = {k: v.data.item() if not isinstance(
                  v, int) else v for k, v in loss_dict.items()}
              t = (time.time() - iter_start_time) / opt.print_freq
              visualizer.print_current_errors(epoch, epoch_iter, errors, t)
              visualizer.plot_current_errors(errors, total_steps)
              #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"])

          # display output images
          if save_fake:
              visuals = OrderedDict([('input_label', util.tensor2label(data['seg_map'][0], opt.label_nc)),
                                    ('synthesized_image', util.tensor2im(
                                        generated.data[0])),
                                    ('real_image', util.tensor2im(data['target'][0]))])
              visualizer.display_current_results(visuals, epoch, total_steps)

          # save latest model
          if total_steps % opt.save_latest_freq == save_delta:
              print('saving the latest model (epoch %d, total_steps %d)' %
                    (epoch, total_steps))
              model.module.save('latest')
              np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

          if epoch_iter >= dataset_size:
              break

      # end of epoch
      iter_end_time = time.time()
      print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

      # save model for this epoch
      if epoch % opt.save_epoch_freq == 0:
          print('saving the model at the end of epoch %d, iters %d' %
                (epoch, total_steps))
          model.module.save('latest')
          model.module.save(epoch)
          np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

  #     ### instead of only training the local enhancer, train the entire network after certain iterations
  #     if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
  #         model.module.update_fixed_params()

      # linearly decay learning rate after certain iterations
      if epoch > opt.niter:
          model.module.update_learning_rate()
if __name__ == '__main__':
    # torch.multiprocessing.set_start_method('spawn')
    main()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: /home/dataset/
debug: False
densepose_nc: 3
display_freq: 100
display_winsize: 512
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 20
instance_feat: False
isTrain: True
label_feat: False
label_nc: 20
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: vfr
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_ce_loss: False
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: fractions.gcd() is deprecated. Use math.gcd() instead.
  


Resuming from epoch 195 at iteration 3536
 #Checking  the dimension and type of data
name of the input and shape -- >  seg_map torch.Size([20, 512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(0.) tensor(1.)
----------------
name of the input and shape -- >  target torch.Size([3, 512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(-1.) tensor(1.)
----------------
name of the input -- >  seg_map_path /content/dataset/train_label/000003_0.png
----------------
name of the input -- >  target_path /content/dataset/train_img/000003_0.png
----------------
name of the input and shape -- >  seg_mask torch.Size([512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.int64 tensor(0) tensor(15)
----------------
#training images = 14220
--------- model used --------- ov_pix2pixHD
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(50, 64, kernel_size=(7, 7), stride=(1, 1))
    (2)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



create web directory /content/drive/My Drive/02_o-viton/source/appearance_generation/checkpoints/vfr/web...


/content/o-viton/appearance_generation/models/ov_pix2pixHD_model.py:179: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  indices = (selected_seg_mask_tensor == int(num_seg_channel)).nonzero()  # nx4


(epoch: 195, iters: 3636, time: 0.216) G_GAN: 1.262 G_VGG: 1.630 D_real: 0.145 D_fake: 0.182 
(epoch: 195, iters: 3736, time: 0.182) G_GAN: 1.488 G_VGG: 1.678 D_real: 0.133 D_fake: 0.144 
(epoch: 195, iters: 3836, time: 0.184) G_GAN: 1.917 G_VGG: 1.534 D_real: 0.085 D_fake: 0.096 
(epoch: 195, iters: 3936, time: 0.182) G_GAN: 1.488 G_VGG: 1.768 D_real: 0.130 D_fake: 0.125 
(epoch: 195, iters: 4036, time: 0.184) G_GAN: 1.818 G_VGG: 1.746 D_real: 0.176 D_fake: 0.271 
(epoch: 195, iters: 4136, time: 0.186) G_GAN: 1.937 G_VGG: 1.149 D_real: 0.164 D_fake: 0.096 
(epoch: 195, iters: 4236, time: 0.186) G_GAN: 1.240 G_VGG: 1.416 D_real: 0.140 D_fake: 0.173 
(epoch: 195, iters: 4336, time: 0.184) G_GAN: 1.076 G_VGG: 1.722 D_real: 0.069 D_fake: 0.264 
(epoch: 195, iters: 4436, time: 0.184) G_GAN: 2.060 G_VGG: 1.914 D_real: 0.122 D_fake: 0.094 
(epoch: 195, iters: 4536, time: 0.184) G_GAN: 2.349 G_VGG: 2.173 D_real: 0.329 D_fake: 0.099 
saving the latest model (epoch 195, total_steps 2763216)
(ep